In [49]:
import torch
from torch import Tensor
import pickle
import tqdm
import pandas as pd
import ast
import numpy as np
print(torch.__version__)
torch.cuda.is_available()
import numpy as np
from scipy import sparse

1.12.1+cu113


In [50]:
def load_pkl():
    # path_to_pickle = '../data/preprocessed/uspt/toy.patent.tsv'
    path_to_pickle = '../data/preprocessed/uspt/patent.tsv.filtered.mt90.ts7'

    teams_pkl = path_to_pickle + '/teams.pkl'
    indexes_pkl = path_to_pickle + '/indexes.pkl'
    teamsvecs_pkl = path_to_pickle + '/teamsvecs.pkl'

    with open(teams_pkl, 'rb') as tb:
        teams = pickle.load(tb)

    with open(indexes_pkl, 'rb') as tb:
        indexes = pickle.load(tb)

    with open(teamsvecs_pkl, 'rb') as tb:
        teamsvecs = pickle.load(tb)

    return teams, indexes, teamsvecs


teams, indexes, teamsvecs = load_pkl()

In [43]:
teamsvecs['skill'][1].nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]))

In [51]:
demo_df = pd.DataFrame(columns=['loc_id', 'loc_name', 'ex_id', 'skills_columns'])
# demo_series = pd.Series()
for tx, team in tqdm.tqdm(enumerate(teams), total=len(teams)):
    try:
        team_skills = list(teamsvecs['skill'][tx].nonzero()[1])
    except KeyError:
        pass
    for ix, each in enumerate(teams[team].members):
        try:
            name = teams[team].members[ix].name
            ex_id = indexes['c2i'][f'{each.id}_{name}']
            loc_name = teams[team].members_details[ix][2]
            loc_id = indexes['l2i'][loc_name]
            demo_series = pd.Series(data={'loc_id':loc_id, 'loc_name':loc_name, 'ex_id':ex_id, 'skills_columns':team_skills})
            # demo_df = demo_df.append(demo_series, ignore_index=True)
            temp_df = pd.DataFrame([{'loc_id':loc_id, 'loc_name':loc_name, 'ex_id':ex_id, 'skills_columns':team_skills}])
            demo_df = pd.concat([demo_df, temp_df], axis=0, ignore_index=True)
        except KeyError:
            pass
        # demo_dict = {'loc_id':loc_id, 'loc_name':loc_name, 'ex_id':ex_id, 'skills_columns':team_skills}
        # break
    # break
demo_df.to_csv('uspt_graph_90_7.csv', index=False)
# demo_df = pd.read_csv('uspt_graph.csv', dtype={'loc_id': np.float64, 'ex_id': np.float64})
# print(demo_df.info())

  0%|          | 0/4608 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not Patent

In [4]:
skills_matrix = torch.load('skills_matrix.pt')

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cuda'


In [44]:
unique_loc_nodes = demo_df['loc_id'].unique()
unique_loc_names = demo_df['loc_name'].unique()
unique_ex_nodes = demo_df['ex_id'].unique()

edge_index_loc_to_expert = torch.stack([torch.from_numpy(demo_df['loc_id'].values.astype('float64')), torch.from_numpy(demo_df['ex_id'].values.astype('float64'))], dim=0)

print('Unique Location Nodes: ', unique_loc_nodes)
print('Total Number of Unique Location Nodes: ', len(unique_loc_nodes))
print()
print('Unique Locations: ', unique_loc_names)
print('Total Number of Unique Locations: ', len(unique_loc_names))
print()
print('Unique Expert Nodes: ', unique_ex_nodes)
print('Total Number of Experts: ', len(unique_ex_nodes))
print()
print('Edge Index Loc to Expert: ', edge_index_loc_to_expert)
print('Shape of Tensor for Edge Index: ', edge_index_loc_to_expert.size())

Unique Location Nodes:  [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 26. 27. 25. 31. 29. 32. 33. 35. 36. 37. 38.
 40. 41. 42. 43. 45. 46. 47. 48. 39. 44. 51. 52. 55. 28. 56. 57. 54. 59.
 60. 58. 30. 61. 63. 50. 64. 65. 66. 70. 68. 69.]
Total Number of Unique Location Nodes:  66

Unique Locations:  ['JP' 'KR' 'DE' 'US' 'TW' 'CN' 'NL' 'IN' 'HU' 'IL' 'FI' 'AT' 'RU' 'SE'
 'SG' 'IT' 'MX' 'GB' 'CH' 'CA' 'IE' 'FR' 'AU' 'DK' 'SA' 'JO' 'ES' 'BE'
 'BR' 'HK' 'DZ' 'CZ' 'CY' 'PL' 'KE' 'VN' 'AR' 'RO' 'PT' 'TH' 'ZA' 'UA'
 'CL' 'TR' 'EG' 'CR' 'DT' 'ID' 'BG' 'NO' 'LI' 'PH' 'LU' 'IR' 'GR' 'PK'
 'NZ' 'DD' 'AE' 'MY' 'GE' 'KY' 'SK' 'TT' 'BB' 'JE']
Total Number of Unique Locations:  66

Unique Expert Nodes:  [0.0000e+00 1.0000e+00 2.0000e+00 ... 1.3628e+04 1.3629e+04 1.3630e+04]
Total Number of Experts:  13631

Edge Index Loc to Expert:  tensor([[  0.,   0.,   0.,  ...,   3.,   3.,   3.],
        [  0.,   1.,   2.,  ..., 681., 682., 679.]], dtype=torc

In [45]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()

# Save node indices:
data["location"].node_id = torch.Tensor(unique_loc_nodes.astype('int8')).type(torch.LongTensor)
data["experts"].node_id = torch.Tensor(demo_df['ex_id'].astype('int8')).type(torch.LongTensor)
# data["location"].node_id = torch.Tensor(unique_loc_nodes.astype('float64'))
# data["experts"].node_id = torch.Tensor(unique_ex_nodes.astype('float64'))

# Add the node features and edge indices:
data["experts"].x = skills_matrix.type(torch.int8).type(torch.LongTensor)
data["location", "of", "experts"].edge_index = edge_index_loc_to_expert.type(torch.int8).type(torch.LongTensor)

# For Message Passing in both the directions
transform = T.Compose([T.ToUndirected()])
data = transform(data)

print(data)

HeteroData(
  location={ node_id=[66] },
  experts={
    node_id=[629521],
    x=[24, 111]
  },
  (location, of, experts)={ edge_index=[2, 629521] },
  (experts, rev_of, location)={ edge_index=[2, 629521] }
)


In [26]:
torch.save(data, 'uspt_hetero.pt')

In [125]:
dummy_skills = demo_df['skills_columns'][0]
total_skills = teamsvecs['skill'].shape[1]
dummy_array = np.zeros([1,total_skills], dtype=float)
dummy_array[0, dummy_skills] = 1
dummy_sparse = sparse.lil_matrix(dummy_array)
dummy_sparse

<1x69679 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in List of Lists format>

In [27]:
x = ast.literal_eval(demo_df['skills_columns'])
x

ValueError: malformed node or string: 0                            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1                            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2                            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
3                            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...
                                ...                        
629516    [1939, 5335, 5337, 5402, 7638, 15937, 21152, 2...
629517                  [12446, 14458, 14460, 14462, 23855]
629518                  [12446, 14458, 14460, 14462, 23855]
629519                  [12446, 14458, 14460, 14462, 23855]
629520                  [12446, 14458, 14460, 14462, 23855]
Name: skills_columns, Length: 629521, dtype: object

In [9]:
print('Skills Matrix Shape, X: ', len(demo_df['skills_columns'])) #629521
print('Skills Matrix Shape, Y: ',teamsvecs['skill'].shape[1]) #69679

Skills Matrix Shape, X:  24
Skills Matrix Shape, Y:  111


In [47]:
# total_skills = teamsvecs['skill'].shape[1]
# skills_df = pd.DataFrame(columns=['skx', 'skills'])
# skills_list = list()
print('Creating Matrix')
skills_matrix = torch.zeros(size=(629521, 69679), dtype=bool)
print(skills_matrix.shape)
print('Dr. Fani')
for sx, sk in tqdm.tqdm(enumerate(demo_df['skills_columns']), total=len(demo_df['skills_columns'])):
    skill_zeros = torch.from_numpy(np.zeros([1, 69679], dtype=bool))
    sk = ast.literal_eval(sk)
    # print(skill_zeros.shape)
    skill_zeros[0, sk] = 1
    # skills_list.append(skill_zeros)
    skills_matrix[sx, :] = skill_zeros
    # break
    # skills_temp = pd.DataFrame({'skx':skx, 'skills':skill_zeros.tolist()})
    # skills_df = pd.concat([skills_df, skills_temp], axis=0, ignore_index=True)
    # break
# skills_sparse = sparse.lil_matrix(sk_arr)
print(skills_matrix.shape)


Creating Matrix
torch.Size([629521, 69679])
Dr. Fani


100%|██████████| 629521/629521 [03:54<00:00, 2679.24it/s] 


torch.Size([629521, 69679])


In [48]:
# torch.save(skills_matrix, 'skills_matrix.pt')
torch.save(skills_matrix, 'skills_matrix_90_5.pt')

In [33]:
first_sk = skills_list[0]
second_sk = skills_list[1]

NameError: name 'skills_list' is not defined

In [28]:
first_sk

NameError: name 'first_sk' is not defined

In [29]:
torch.save(skills_list,'skills_list.pt')

NameError: name 'skills_list' is not defined

In [21]:
stacked_skills = torch.cat(skills_list)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 43864393759 bytes.

In [ ]:
stacked_skills.size()

In [ ]:
torch.save(stacked_skills, 'stacked_skills.pt')

In [31]:
torch.from_numpy(skills_list[0]).to(torch.float)

tensor([[1., 1., 1.,  ..., 0., 0., 0.]])

In [ ]:
dummy_sparse.data

In [129]:
dummy_array.nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64))

In [ ]:
total_skills = teamsvecs['skill'].shape[1]
total_skills

In [10]:
uspt_path = 'link_df_75_3_skills.csv'

print('uspt:')
print('===========')
uspt_df = pd.read_csv(uspt_path)
uspt_df

uspt:


,loc_id,loc_name,ex_id,skills
0,0,JP,0,H01L41/297B41J2/14233H01L41/0471B41J2/14201B41...
1,0,JP,1,H01L41/297B41J2/14233H01L41/0471B41J2/14201B41...
2,0,JP,2,H01L41/297B41J2/14233H01L41/0471B41J2/14201B41...
3,0,JP,3,H01L41/297B41J2/14233H01L41/0471B41J2/14201B41...
4,0,JP,4,B41J15/005B41J15/04B65H29/60B65H35/0006B41J29/...
...,...,...,...,...
629458,3,US,608,B05D3/0254C08G12/043B05D1/02C08G12/08C09D161/2...
629459,3,US,680,B25J15/0633B25J15/0625B25J15/0691B25J15/0658B6...
629460,3,US,681,B25J15/0633B25J15/0625B25J15/0691B25J15/0658B6...
629461,3,US,682,B25J15/0633B25J15/0625B25J15/0691B25J15/0658B6...


In [35]:
transform = T.RandomLinkSplit(
    num_val=0.10,
    num_test=0.10,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("location", "of", "experts"),
    rev_edge_types=("experts", "rev_of", "location"),
)

print('Data Before Splitting', data)
print("==============")
print()
train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)

Data Before Splitting HeteroData(
  location={ node_id=[3] },
  experts={
    node_id=[24],
    x=[24, 111]
  },
  movie={},
  (location, of, experts)={ edge_index=[2, 24] },
  (experts, rev_of, location)={ edge_index=[2, 24] }
)

Training data:
HeteroData(
  location={ node_id=[3] },
  experts={
    node_id=[24],
    x=[24, 111]
  },
  movie={},
  (location, of, experts)={
    edge_index=[2, 14],
    edge_label=[6],
    edge_label_index=[2, 6]
  },
  (experts, rev_of, location)={ edge_index=[2, 14] }
)

Validation data:
HeteroData(
  location={ node_id=[3] },
  experts={
    node_id=[24],
    x=[24, 111]
  },
  movie={},
  (location, of, experts)={
    edge_index=[2, 20],
    edge_label=[6],
    edge_label_index=[2, 6]
  },
  (experts, rev_of, location)={ edge_index=[2, 20] }
)


In [36]:
from torch_geometric.loader import LinkNeighborLoader

edge_label_index = train_data["location", "of", "experts"].edge_label_index
edge_label = train_data["location", "of", "experts"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("location", "of", "experts"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  location={ node_id=[3] },
  experts={
    node_id=[22],
    x=[22, 111]
  },
  movie={},
  (location, of, experts)={
    edge_index=[2, 14],
    edge_label=[18],
    edge_label_index=[2, 18],
    input_id=[6]
  },
  (experts, rev_of, location)={ edge_index=[2, 14] }
)


In [22]:
import torch
from torch import Tensor
from torch_geometric.data import download_url, extract_zip
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
import tqdm
import torch.nn.functional as F

In [38]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # Define a 2-layer GNN computation graph.
        # Use a *single* `ReLU` non-linearity in-between.
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.movie_lin = torch.nn.Linear(20, hidden_channels)
        self.user_emb = torch.nn.Embedding(data["location"].num_nodes, hidden_channels)
        self.movie_emb = torch.nn.Embedding(data["experts"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
            "location": self.user_emb(data["location"].node_id),
            "experts": self.movie_lin(data["experts"].x) + self.movie_emb(data["experts"].node_id),
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["location"],
            x_dict["experts"],
            data["location", "of", "experts"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=64)

print(model)

NotImplementedError: 

In [ ]:
import torch
from torch import Tensor
from torch_geometric.data import download_url, extract_zip
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
import tqdm
import torch.nn.functional as F